### Step 1. Initialization of libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss


### Step 2. Reading dataset

In [2]:
data=pd.read_csv("/home/bscuser/data/train.csv")

In [3]:
data

,Sequence,Active
0,DKWL,0
1,FCHN,0
2,KDQP,0
3,FNWI,0
4,NKRM,0
...,...,...
111995,GSME,0
111996,DLPT,0
111997,SGHC,0
111998,KIGT,0


### Step 3. Preparing data and One Hot Encoding.

Taking into account that Protein molecules are a sequence, we have to split it for encoding every part of the sequence. Next comes the splitting  of thesequence by letters and using the One Hot Encoding approach. For encoding we used the One Hot encoder from the sklearn package. As a result, we received an array with 26 encoded categories for each row of the input data. 

In [4]:
def split(word): 
    return [char for char in word]

In [5]:
first=[]
second=[]
third=[]
fourth=[]
for index,row in data.iterrows():
    first.append(split(row[0])[0])
    second.append(split(row[0])[1])
    third.append(split(row[0])[2])
    fourth.append(split(row[0])[3])

In [6]:
data_split=pd.DataFrame()
data_split["First"]=first
data_split["Second"]=second
data_split["Third"]=third
data_split["Fourth"]=fourth
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(data_split)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

In [22]:
enc.categories_

[array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)]

In [8]:
enc_data=enc.transform(data_split).toarray()

In [10]:
enc_data.shape

(112000, 80)

### Step 4. Splitting dataset for training and validation. 

For calculation F1 metric dataset was split into train and validation subsets 

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
X=enc_data
y=np.array(data["Active"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

### Step 5. Training of model

Taking into account the conditions of task SVC was chosen as the main approach.


In [23]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_dim=80, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=150, batch_size=256)

Epoch 1/150
100800/100800 [==============================] - 1s 11us/step - loss: 0.2204 - acc: 0.9500
Epoch 2/150
100800/100800 [==============================] - 1s 7us/step - loss: 0.0824 - acc: 0.9649
Epoch 3/150
100800/100800 [==============================] - 1s 7us/step - loss: 0.0640 - acc: 0.9752
Epoch 4/150
100800/100800 [==============================] - 1s 7us/step - loss: 0.0593 - acc: 0.9773
Epoch 5/150
100800/100800 [==============================] - 1s 7us/step - loss: 0.0547 - acc: 0.9790
Epoch 6/150
100800/100800 [==============================] - 1s 7us/step - loss: 0.0491 - acc: 0.9813
Epoch 7/150
100800/100800 [==============================] - 1s 7us/step - loss: 0.0444 - acc: 0.9832
Epoch 8/150
100800/100800 [==============================] - 1s 7us/step - loss: 0.0408 - acc: 0.9846
Epoch 9/150
100800/100800 [==============================] - 1s 7us/step - loss: 0.0382 - acc: 0.9853
Epoch 10/150
100800/100800 [==============================] - 1s 7us/step - loss:

In [25]:
model.evaluate(X_val,y_val)

11200/11200 [==============================] - 0s 29us/step


[0.02042468285065545, 0.9925]

### Step 6. Calculation of metric

As a metric for the model we chose the F1 score. The score was calculated on the validation dataset.

In [29]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
y_pred=model.predict_classes(X_val)
f1_score(y_val, y_pred, average='macro')

0.9462028030809874

### Step 7. Reading and encoding dataset. Prediction of classes. 

The same endocder was used for training dataset. Predicted classes were saved in CSV file. 

In [34]:
test_data=pd.read_csv("/home/bscuser/data/test.csv")

In [35]:
first=[]
second=[]
third=[]
fourth=[]
for index,row in test_data.iterrows():
    first.append(split(row[0])[0])
    second.append(split(row[0])[1])
    third.append(split(row[0])[2])
    fourth.append(split(row[0])[3])

In [36]:
test_data_split=pd.DataFrame()
test_data_split["First"]=first
test_data_split["Second"]=second
test_data_split["Third"]=third
test_data_split["Fourth"]=fourth

In [37]:
enc_test_data=enc.transform(test_data_split).toarray()

In [39]:
y_test=model.predict_classes(enc_test_data)

In [40]:
np.savetxt("test_answer_DL.csv", y_test, delimiter=",")